This method is a post-processing applied to some of our team's models.

This notebook incorporated geojson into [Saito's post-processing](https://www.kaggle.com/saitodevel01/indoor-post-processing-by-cost-minimization) and solved it as a nonlinear cost minimization problem.

To combine Saito's post-processing with geojson, I defined cost function as follows,
$$
L(X_{1:N}) = \sum_{i=1}^{N} \alpha_i \| X_i - \hat{X}_i \|^2 + \sum_{i=1}^{N-1} \beta_i \| (X_{i+1} - X_{i}) - \Delta \hat{X}_i \|^2 + \sum_{i=1}^{N-1} \ \sum_{j=1}^{M} \gamma_{ij} \| Dist(X_{ij}) \| ^2
$$
where $X_{ij}$ is an evenly spaced point on the line segment $X_{i}$$X_{i+1}$, and $Dist(X_{ij})$ represents the distance to the wall, which is represented as the following distance image created from geojson.

In [ ]:
import os
from glob import glob
import numpy as np
import math
import pandas as pd
from tqdm.notebook import tqdm
import copy
import pickle as pkl
import json
import matplotlib.pyplot as plt
import multiprocessing

from sklearn.model_selection import GroupKFold
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import scipy

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW, lr_scheduler

import gc
import warnings
warnings.filterwarnings('ignore')

import glob
from scipy import optimize
from scipy.optimize import minimize
import math

In [ ]:
site = '5d27096c03f801723c31e5e0'
floorNo = 'F3'

floor = plt.imread(f"../input/indoor-location-navigation/metadata/{site}/{floorNo}/floor_image.png")
bw = plt.imread(f"../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_bw_filled_x16.png")
dist = plt.imread(f"../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_cv_dist_normalized_x16.png")

plt.figure(figsize=(18,12))
plt.subplot(1,3,1)
plt.title('Map')
plt.imshow(floor)
plt.subplot(1,3,2)
plt.title('Wall')
plt.imshow(bw[::-1,:])
plt.subplot(1,3,3)
plt.title('Dist')
plt.imshow(dist[::-1,:])
plt.show()

To make it easier to converge to the correct position, the scale of the distance image is multiplied by 16.

Since this equation is nonlinear, it can be solved using the COBYLA method of scipy's minimize function.

The raw prediction from machine learning is based on [Tri's transformer model](https://www.kaggle.com/shinomoriaoshi/iln-transformer-inference).



* public score : 5.17309 -> 4.38225
* private score : 5.50094 -> 4.95899

In [ ]:
train_waypoints = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
ss = pd.read_csv('../input/indoorlocationnavigationbwdist/metadata/Tri_raw_prediction.csv')

tmp = ss['site_path_timestamp'].apply(lambda s : pd.Series(s.split('_')))
ss['site'] = tmp[0]
ss['path'] = tmp[1]
ss['timestamp'] = tmp[2].astype(float)

ss = ss.merge(train_waypoints[['site','floor','floorNo']].drop_duplicates())

# Saito's post-processing with geojson

In [ ]:
!git clone --depth 1 https://github.com/location-competition/indoor-location-competition-20 indoor_location_competition_20
!rm -rf indoor_location_competition_20/data

from indoor_location_competition_20.io_f import read_data_file
import indoor_location_competition_20.compute_f as compute_f

In [ ]:
def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

In [ ]:
def calc_points(x1_realnum, y1_realnum, x2_realnum, y2_realnum):
    
    x1 = math.floor(x1_realnum)
    y1 = math.floor(y1_realnum)
    x2 = math.floor(x2_realnum)
    y2 = math.floor(y2_realnum)
    
    points = []
    
    if (x1 == x2) and (y1 == y2):
        points.append([x1_realnum,y1_realnum])
        points.append([x2_realnum,y2_realnum])
        return np.array(points)

    x1, y1 = x1_realnum, y1_realnum
    x2, y2 = x2_realnum, y2_realnum
    
    step_x = np.sign(x2 - x1)
    step_y = np.sign(y2 - y1)
    a = (y2-y1)/(x2-x1)
    dx = 1/((a**2+1)**0.5)
    dy = a/((a**2+1)**0.5)
    n = math.floor(((y2-y1)**2 + (x2-x1)**2)**0.5)
    
    x = x1_realnum
    y = y1_realnum
    points.append([x1_realnum, y1_realnum])
    for i in range(n-1):
        x = x + step_x * dx
        y = y + step_y * dy
        points.append([x, y])
    points.append([x2_realnum, y2_realnum])
    
    return np.array(points)

In [ ]:
def f(v,path_df, alpha_, beta_, xy_hat_, delta_xy_hat_, dist, delta_t): 

    length = int(len(v)/2)-1
    dx,dy = v[-2], v[-1]
    x = v[0:-2:2] + dx
    y = v[1:-1:2] + dy
    v = v[0:-2]

    delta_x = delta_xy_hat_[0::2]
    delta_y = delta_xy_hat_[1::2]
    
    cost_alpha = sum(alpha_ * (v - xy_hat_)**2)
    cost_beta = sum(beta_ * (v[2:2*length] - v[0:2*length-2] - delta_xy_hat_)**2)
    
    cost_gamma = 0
    for i in range(length-1):
        points = calc_points(x[i],y[i],x[i+1],y[i+1])
        for j, point in enumerate(points):
            gamma = 100
            if j==0 or j==len(points)-1:
                start_and_end_coeff = 20
            else:
                start_and_end_coeff = 1
            
            if point[0] < 0 or point[1] < 0 or point[0] > dist.shape[1] or point[1] > dist.shape[0]:
                cost_gamma += gamma * start_and_end_coeff
            else:
                try:
                    px = point[0]
                    py = point[1]
                    pxf = math.floor(px)
                    pyf = math.floor(py)
                    dist_bilinear = (pxf+1-px)*((pyf+1-py)*dist[pyf][pxf]+(py-pyf)*dist[pyf+1][pxf])+(px-pxf)*((pyf+1-py)*dist[pyf][pxf+1]+(py-pyf)*dist[pyf+1][pxf+1])
                    cost_gamma += start_and_end_coeff * gamma*dist_bilinear
                except:
                    cost_gamma += gamma * start_and_end_coeff
        
    return cost_alpha + cost_beta + cost_gamma

In [ ]:
show_path_list = ['07db4eec1fc8df5040d86602','0412d582bb8a2c89400a1ffb', '08a29ac71fe460a44ca73b38', '0969b6db91bb1697f99a8a21', '0fb75bf676e1cddace3590d5', '1747b3e80ba80910158c0cef', '1e85728385bc83e6bb650798', '270a66500675e14fb3154c63','947e17f82dbddfbdb4cb2447']

In [ ]:
def correct_path(args):
    path, path_df = args
    site = path_df['site'].iloc[0]
    floorNo = path_df['floorNo'].iloc[0]
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    example_path = glob.glob(f'../input/indoor-location-navigation/**/{path}.txt', recursive=True)
    example = read_data_file(example_path[0])
    rel_positions = compute_rel_positions(example.acce, example.ahrs)

    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    
    try:
        delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3],
        axis=0), axis=0)(T_ref), axis=0)
    except:
        return path_df.drop(['site','path','timestamp'],axis=1)
    
    if sum(delta_xy_hat.flatten()) == 0:
        return path_df.drop(['site','path','timestamp'],axis=1)
    
    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (8.1)**(-2) * np.ones(N)
    beta  = (0.4 + 0.4 * 1e-3 * delta_t)**(-2)
    
    alpha_ = np.ravel(np.stack([alpha, alpha],1))
    beta_ = np.ravel(np.stack([beta, beta],1))
    
    xy_hat_ = np.ravel(xy_hat)
    delta_xy_hat_ = np.ravel(delta_xy_hat)
    
    dist = plt.imread(f"../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_cv_dist_normalized_x16.png")
    
    x = xy_hat_[0::2]
    y = xy_hat_[1::2]
    
    initial_v = np.ravel(path_df[['x', 'y']].values)
    initial_v = np.append(initial_v, [0,0])
    
    v = minimize( f, x0=initial_v*16, args=(path_df, alpha_, beta_, xy_hat_*16, delta_xy_hat_*16, dist, delta_t), method="cobyla",options={'rhobeg': 10.0, 'maxiter': 1000, 'disp': False, 'catol': 0.0002})
    
    cost = f(v['x']*16,path_df, alpha_, beta_, xy_hat_*16, delta_xy_hat_*16, dist, delta_t)
    
    dx,dy = v['x'][-2], v['x'][-1]
    x_ = v['x'][0:-2:2] + dx
    y_ = v['x'][1:-1:2] + dy
    
    # visualize 
    if path in show_path_list:
        print(site,floorNo,path,cost, flush=True)
        plt.figure(figsize=(12,12))
        plt.title(f'{site}, {floorNo}, {path}, {cost}')
        plt.imshow(dist)
        plt.plot(x*16, y*16, '.-',label=f"before_pp")
        plt.plot(x_, y_, '.-',label=f"after_pp")
        plt.plot(dist.shape[1]/2 + np.cumsum(np.append(0,delta_xy_hat[:,0])*16), dist.shape[0]/2+ np.cumsum(np.append(0,delta_xy_hat[:,1])*16), '.-',label="delta")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
        plt.show()
    
    return pd.DataFrame({
        'site_path_timestamp' : path_df['site_path_timestamp'],
        'floor' : path_df['floor'],
        'x' : x_/16,
        'y' : y_/16
    })

To compare delta and prediction, the shape of delta is shown in the center of the image.

In [ ]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes = processes) as pool:
    dfs = pool.imap_unordered(correct_path, ss.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
ss = pd.concat(dfs).sort_values('site_path_timestamp')

# Submission

In [ ]:
ss.to_csv('submission.csv', index = None)